In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('Hiring').getOrCreate()

21/09/11 10:37:30 WARN Utils: Your hostname, Subhajits-MacBook-Air.local resolves to a loopback address: 127.0.0.1, but we couldn't find any external IP address!
21/09/11 10:37:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/09/11 10:37:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/09/11 10:37:33 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: 30:c6:7e:ee:36:d9:70:ce


In [9]:
training = spark.read.csv('/Users/subhajitsaha/Documents/Data Science Projects/Salary prediction/hiring.csv',
                         header=True, inferSchema=True)

In [10]:
training.show()

+---------+----------+---------------+------+
|experince|test_score|interview_score|salary|
+---------+----------+---------------+------+
|        0|         8|              9| 50000|
|        0|         8|              9| 45000|
|        5|         6|              7| 60000|
|        2|        10|             10| 65000|
|        7|         9|              6| 70000|
|        3|         7|             10| 62000|
|       10|         8|              7| 72000|
|       11|         7|              8| 80000|
+---------+----------+---------------+------+



In [11]:
training.printSchema()

root
 |-- experince: integer (nullable = true)
 |-- test_score: integer (nullable = true)
 |-- interview_score: integer (nullable = true)
 |-- salary: integer (nullable = true)



In [12]:
training.columns

['experince', 'test_score', 'interview_score', 'salary']

In [13]:
# we have to group all the independent features and create a single new feature with them
# [experince, test_score, interview_score]
from pyspark.ml.feature import VectorAssembler
feature_assembler = VectorAssembler(inputCols=['experince', 'test_score', 'interview_score'], outputCol='Independent features')

In [14]:
output = feature_assembler.transform(training)

In [15]:
output.show()

+---------+----------+---------------+------+--------------------+
|experince|test_score|interview_score|salary|Independent features|
+---------+----------+---------------+------+--------------------+
|        0|         8|              9| 50000|       [0.0,8.0,9.0]|
|        0|         8|              9| 45000|       [0.0,8.0,9.0]|
|        5|         6|              7| 60000|       [5.0,6.0,7.0]|
|        2|        10|             10| 65000|     [2.0,10.0,10.0]|
|        7|         9|              6| 70000|       [7.0,9.0,6.0]|
|        3|         7|             10| 62000|      [3.0,7.0,10.0]|
|       10|         8|              7| 72000|      [10.0,8.0,7.0]|
|       11|         7|              8| 80000|      [11.0,7.0,8.0]|
+---------+----------+---------------+------+--------------------+



In [16]:
output.columns

['experince',
 'test_score',
 'interview_score',
 'salary',
 'Independent features']

In [17]:
finalized_data = output.select('Independent features', 'salary')
finalized_data.show()

+--------------------+------+
|Independent features|salary|
+--------------------+------+
|       [0.0,8.0,9.0]| 50000|
|       [0.0,8.0,9.0]| 45000|
|       [5.0,6.0,7.0]| 60000|
|     [2.0,10.0,10.0]| 65000|
|       [7.0,9.0,6.0]| 70000|
|      [3.0,7.0,10.0]| 62000|
|      [10.0,8.0,7.0]| 72000|
|      [11.0,7.0,8.0]| 80000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression

In [19]:
# train-test split
train_data, test_data = finalized_data.randomSplit([0.8, 0.2])

In [20]:
regressor = LinearRegression(featuresCol='Independent features', labelCol='salary')

In [23]:
regressor = regressor.fit(train_data)

21/09/11 11:04:25 WARN Instrumentation: [6e1e9e61] regParam is zero, which might cause numerical instability and overfitting.


In [26]:
regressor.coefficients

DenseVector([3025.967, 2409.6277, 1540.8483])

In [27]:
regressor.intercept

16334.859621641037

In [28]:
pred_result = regressor.evaluate(test_data)

In [29]:
pred_result.predictions.show()

+--------------------+------+-----------------+
|Independent features|salary|       prediction|
+--------------------+------+-----------------+
|       [0.0,8.0,9.0]| 50000|49479.51560428971|
|      [3.0,7.0,10.0]| 62000|57688.63718520323|
+--------------------+------+-----------------+



In [30]:
pred_result.meanAbsoluteError, pred_result.meanSquaredError

(2415.9236052535307, 9429376.663495112)